In [16]:
!nvidia-smi

Wed May  5 21:39:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    33W / 250W |   1155MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory


In [18]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 99% 1.02G/1.04G [00:13<00:00, 75.5MB/s]
100% 1.04G/1.04G [00:13<00:00, 80.5MB/s]


In [19]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

fatal: destination path 'Image-Captioning' already exists and is not an empty directory.


In [20]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [22]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 10  # number of epochs to train for (if early stopping is not triggered)
batch_size = 265
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [24]:
DATA_NAME = 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/data.json'
IMGS_PATH = 'flickr8k/images/'

In [25]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 40000/40000 [00:00<00:00, 301850.01it/s]


In [26]:
len(vocab)

5089

In [27]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

In [28]:
t_params

{'batch_size': 265,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 10,
 'fine_tune_encoder': False,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd545b3c10>,
 'workers': 2}

In [29]:
fit(t_params=t_params, m_params=m_params)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/114]	Batch Time 7.710 (7.710)	Data Load Time 3.707 (3.707)	Loss 9.4432 (9.4432)	Top-5 Accuracy 0.063 (0.063)
Epoch: [0][100/114]	Batch Time 1.410 (1.981)	Data Load Time 0.000 (0.595)	Loss 4.7509 (5.4879)	Top-5 Accuracy 54.977 (44.968)
-------------------- Validation --------------------
Validation: [0/19]	Batch Time 5.143 (5.143)	Loss 5.4544 (5.4544)	Top-5 Accuracy 50.632 (50.632)	
----- Bleu-n Scores -----
1: 56.66132822995774
2: 30.55702677997155
3: 15.027989270660106
4: 7.251530933679744
-------------------------

 * LOSS - 5.462, TOP-5 ACCURACY - 50.351, BLEU-4 - 0.0734046482414005

__________________________________________________
-----------

In [30]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2.pth.tar /content/drive/MyDrive/weights/

In [33]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2.pth.tar'
epochs = 15

t_params = {
    'data_name': DATA_NAME+'_finetune',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2_finetune',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 15,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd545b3c10>,
 'workers': 2}

In [34]:
fit(t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 6
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [6][0/469]	Batch Time 3.324 (3.324)	Data Load Time 0.923 (0.923)	Loss 3.9871 (3.9871)	Top-5 Accuracy 63.707 (63.707)
Epoch: [6][100/469]	Batch Time 0.712 (0.735)	Data Load Time 0.000 (0.010)	Loss 3.8380 (3.8848)	Top-5 Accuracy 65.793 (66.213)
Epoch: [6][200/469]	Batch Time 0.701 (0.721)	Data Load Time 0.001 (0.006)	Loss 3.9145 (3.8768)	Top-5 Accuracy 65.765 (66.371)
Epoch: [6][300/469]	Batch Time 0.720 (0.717)	Data Load Time 0.000 (0.004)	Loss 3.9443 (3.8680)	Top-5 Accuracy 65.971 (66.558)
Epoch: [6][400/469]	Batch Time 0.716 (0.714)	Data Load Time 0.000 (0.004)	Loss 3.8165 (3.8492)	Top-5 Accuracy 69.960 (66.910)
---

In [36]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2_finetune.pth.tar')

Loaded Checkpoint!!
Last Epoch: 10
Best Bleu-4: 0.1607200452757974


In [37]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_2_finetune.pth.tar /content/drive/MyDrive/weights/

In [38]:
!nvidia-smi

Wed May  5 23:29:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    35W / 250W |  10319MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------